In [1]:
import numpy as np
from quant_rotor.models.dense.support_ham import write_matrix_elements as w_m_e

np.set_printoptions(suppress = True, linewidth = 1500, threshold = np.inf, precision = 12)

In [2]:
def free_one_body(i, j, max_m):

    # Free particle on a ring Hamiltonian in the momentum representation.
    # K = \sum_{m=0}^{2l} (m-l)^2 |m><m|
    if i == j:
        return (i-max_m)**2
    else:
        return 0.0

def interaction_two_body_coplanar(i1, i2, j1, j2):

    # returns <i1,i2|V|j1,j2> where V is the dipole-dipole interaction between two planar rotors oriented along the x-axis.
    # See https://arxiv.org/abs/2401.02887 for details.
    # V = 0.75 \sum_{m1, m2} (|m1, m2> <m1+1, m2+1| + |m1, m2> <m1-1, m2-1|) - 0.25 \sum_{m1, m2} (|m1, m2> <m1-1, m2+1| + |m1, m2> <m1+1, m2-1|)
    if ((abs(i1 - j1) != 1) or (abs(i2 - j2) != 1)):
        return 0.0
    elif (i1 == j1 + 1):
        if (i2 == j2 + 1):
            return 0.75
        else:
            return -0.25
    else:
        if (i2 == j2 + 1):
            return -0.25
        else:
            return 0.75

def interaction_yiyj(i1, i2, j1, j2):

    # returns <i1,i2|yiyj|j1,j2> where
    # yiyj = -0.25 \sum_{m1, m2} (|m1, m2> <m1+1, m2+1| + |m1, m2> <m1-1, m2-1|) + 0.25 \sum_{m1, m2} (|m1, m2> <m1-1, m2+1| + |m1, m2> <m1+1, m2-1|)
    if ((abs(i1 - j1) != 1) or (abs(i2 - j2) != 1)):
        return 0.0
    elif (i1 - j1 == i2 - j2):
        return -0.25
    else:
        return 0.25

def interaction_xixj(i1, i2, j1, j2):

    # returns <i1,i2|xixj|j1,j2> where
    # xixj = 0.25 \sum_{m1, m2} (|m1, m2> <m1+1, m2+1| + |m1, m2> <m1-1, m2-1|) + 0.25 \sum_{m1, m2} (|m1, m2> <m1-1, m2+1| + |m1, m2> <m1+1, m2-1|)
    if ((abs(i1 - j1) != 1) or (abs(i2 - j2) != 1)):
        return 0.0
    else:
        return 0.25

def interaction_cross_terms(i1, i2, j1, j2):

    # returns <i1,i2|xiyj + yixj|j1,j2> where
    # xiyj + yixj = 0.5i \sum_{m1,m2} (|m1, m2> <m1+1, m2+1| - |m1, m2> <m1-1, m2-1|)
    if ((i1 - j1) == 1) and (i2 - j2 == 1):
        return -0.5*1J
    elif ((i1 - j1) == -1) and (i2 - j2 == -1):
        return 0.5*1J
    else:
        return 0.0

In [3]:
def interaction_general_angle(i1, i2, j1, j2, tau):

    v_xi_xj = interaction_xixj(i1, i2, j1, j2)
    v_yi_yj = interaction_yiyj(i1, i2, j1, j2)

    return v_yi_yj + (1.0 - 3.0*(np.cos(tau))**2)*v_xi_xj

In [4]:
def write_matrix_elements(m_max, gamma):
    d = 2*m_max + 1
    K = np.zeros((d,d))
    for i in range(d):
        for j in range(i,d):
            K[i,j] = free_one_body(i, j, m_max)

    V = np.zeros((d**2, d**2))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                for l in range(d):
                    # only want to write the upper triangular parts
                    # if k*d + l >= i*d + j:
                    #if hg.interaction_two_body_coplanar(i,j,k,l) != 0:
                        V[i*d + j, k*d + l] = interaction_general_angle(i,j,k,l, gamma)

    return K, V

In [37]:
state = 33
gamma = np.pi / 3

In [38]:
K_o, V_o = write_matrix_elements((state-1)//2, gamma)
K_n, V_n = w_m_e((state-1)//2, gamma)

In [39]:
np.array_equal(K_o, K_n)

True

In [40]:
np.array_equal(V_o, V_n)

True